In [5]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model
#use tf anaconda env

In [6]:
# initialize mediapipe
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

### Importing the Models

In [7]:
example_model = tf.keras.models.load_model('C:\Projects\Hand gesture Recogonition\mp_hand_gesture')
model = load_model('C:\Projects\Hand gesture Recogonition')

### Getting the labels

In [8]:
example_gestures = open('gesture.names', 'r')
example_classNames = example_gestures.read().split('\n')
example_gestures.close()
print(example_classNames)

gestures = open('labels.names', 'r')
classNames = gestures.read().split('\n')
gestures.close()
print(classNames)

['okay', 'peace', 'thumbs up', 'thumbs down', 'call me', 'stop', 'rock', 'live long', 'fist', 'smile']
['paper', 'rock', 'scissors']


### Running the Web camera
#### Model we trained

In [9]:
cap = cv2.VideoCapture(0)

while True:
    # Read each frame from the webcam
    _, frame = cap.read()
    x, y, _ = frame.shape
    
    # Flip the frame vertically
    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Get hand landmark prediction
    result = hands.process(framergb)
    # print(result.multi_hand_landmarks)
    className = ''

    # Post-process the result
    if result.multi_hand_landmarks:
        landmarks = []
        for handslms in result.multi_hand_landmarks:
            for lm in handslms.landmark:
                # print(id, lm)
                lmx = int(lm.x * x)
                lmy = int(lm.y * y)

                landmarks.append([lmx, lmy])
            
            # Drawing landmarks on frames
            mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)

            # Resize the image to match the input shape of your model
            resized_frame = cv2.resize(framergb, (224, 224))

            # Make a prediction
            prediction = model.predict(np.expand_dims(resized_frame, axis=0))
            
            # Get the class label
            classID = np.argmax(prediction)
            className = classNames[classID]

            confidence = prediction[0][classID]
            confidence_percentage = confidence * 100
            text = f"{className}: {confidence_percentage:.2f}%"

        # Show the prediction on the frame
        cv2.putText(frame, text,(10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    # Show the final output
    cv2.imshow("Output", frame)
    if cv2.waitKey(1) == ord('q'):
        break

# Release the webcam and destroy all active windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 237ms/step


### Model gotten from https://techvidvan.com/tutorials/hand-gesture-recognition-tensorflow-opencv/

In [10]:
cap = cv2.VideoCapture(0)

while True:
  # Read each frame from the webcam
  _, frame = cap.read()
  x , y, c = frame.shape
  # Flip the frame vertically
  frame = cv2.flip(frame, 1)
  framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

  # Get hand landmark prediction
  result = hands.process(framergb)

  example_className = ''

  # post process the result
  if result.multi_hand_landmarks:
    landmarks = []
    for handslms in result.multi_hand_landmarks:
        for lm in handslms.landmark:
            # print(id, lm)
            lmx = int(lm.x * x)
            lmy = int(lm.y * y)

            landmarks.append([lmx, lmy])
        # Drawing landmarks on frames
        mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)

        # input_shape = (150, 150, 3)
        # input_image = np.zeros(input_shape, dtype=np.uint8)

        # # Place landmarks on the image
        # for landmark in landmarks:
        #     x, y = landmark
        #     # Ensure the coordinates are within the image boundaries
        #     x = min(max(x, 0), input_shape[1] - 1)
        #     y = min(max(y, 0), input_shape[0] - 1)
        #     # Set a pixel value for the landmark
        #     input_image[y, x, :] = 255
            
        # input_image = np.expand_dims(input_image, axis=0)

        prediction = example_model.predict([landmarks])
        # print(prediction)
        classID = np.argmax(prediction)
        example_className = example_classNames[classID]

        confidence = prediction[0][classID]
        confidence_percentage = confidence * 100
        text = f"{example_className}: {confidence_percentage:.2f}%"

        # Show the prediction on the frame
        cv2.putText(frame, text,(10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

      # Show the final output
  cv2.imshow("Output", frame)
  if cv2.waitKey(1) == ord('q'):
    break

# release the webcam and destroy all active windows
cap.release()
cv2.destroyAllWindows()